In [2]:
!git commit -am "WIP refactored code"
!git push

On branch binder
Your branch is up to date with 'origin/binder'.

Untracked files:
	../.bash_logout
	../.bashrc
	../.cache/
	../.conda/
	../.ipython/
	../.local/
	../.profile
	../.wget-hsts

nothing added to commit but untracked files present
Username for 'https://github.com': ^C


In [3]:
!pip install  lxml pandas
from urllib.request import urlopen
from lxml import html, etree
import pandas as pd
import numpy as np
import json

    100% |████████████████████████████████| 5.7MB 119kB/s eta 0:00:01
    100% |████████████████████████████████| 11.7MB 56kB/s  eta 0:00:01   33% |██████████▋                     | 3.9MB 39.4MB/s eta 0:00:01
    100% |████████████████████████████████| 12.2MB 53kB/s  eta 0:00:01
  Found existing installation: numpy 1.7.0
    Uninstalling numpy-1.7.0:
      Successfully uninstalled numpy-1.7.0
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
def extractText(ele):
    return ele.text.strip()

In [5]:
def extractHoldingsFromStock(url):
    print(url)
    page=urlopen(url)
    tree = html.fromstring(page.read().decode('utf-8'))
    holding=tree.xpath('//div[@class="holding"]')[0]
    tbody=holding.xpath('//tbody')[0]
    holding={
        'FullName':tbody.xpath('//div[@data-type="full"]'),
        'Abbreviation':tbody.xpath('//div[@data-type="abbreviation"]'),
        'Percentage':tbody.xpath('//td[@data-type="percent"]')
    }
    df = pd.DataFrame(data=holding)
    df=df.apply(lambda s:s.apply(extractText))
    return df

In [39]:
funds=[]
funds.append({
        'Name':'Janus Henderson Global Technology Fund',
        'URL':'https://www.bloomberg.com/quote/JAGTX:US',
        'Allocation':4
    })
funds.append({
        'Name':'iShares Healthcare Innovation UCITS ETF',
        'URL':'https://www.bloomberg.com/quote/HEAL:SW',
        'Allocation':2
    })
funds.append({
        'Name':'Lyxor MSCI World Information Technology ',
        'URL':'https://www.bloomberg.com/quote/TNOW:IM',
        'Allocation':2
    })


In [40]:
df=pd.DataFrame(funds,columns=['Name','URL','Allocation'])
df

,Name,URL,Allocation
0,Janus Henderson Global Technology Fund,https://www.bloomberg.com/quote/JAGTX:US,4
1,iShares Healthcare Innovation UCITS ETF,https://www.bloomberg.com/quote/HEAL:SW,2
2,Lyxor MSCI World Information Technology,https://www.bloomberg.com/quote/TNOW:IM,2


In [41]:
df['Holdings']=df.URL.apply(extractHoldingsFromStock)

https://www.bloomberg.com/quote/JAGTX:US
https://www.bloomberg.com/quote/HEAL:SW
https://www.bloomberg.com/quote/TNOW:IM


In [42]:
sum=df.Allocation.sum()
weights=list(map(lambda v:round(v/sum,2),df.Allocation))
df['OverallWeight']=weights
df[['Name','OverallWeight']]

,Name,OverallWeight
0,Janus Henderson Global Technology Fund,0.50
1,iShares Healthcare Innovation UCITS ETF,0.25
2,Lyxor MSCI World Information Technology,0.25


In [43]:
df

,Name,URL,Allocation,Holdings,OverallWeight
0,Janus Henderson Global Technology Fund,https://www.bloomberg.com/quote/JAGTX:US,4,FullName Abbreviation...,0.50
1,iShares Healthcare Innovation UCITS ETF,https://www.bloomberg.com/quote/HEAL:SW,2,FullName Abbreviation...,0.25
2,Lyxor MSCI World Information Technology,https://www.bloomberg.com/quote/TNOW:IM,2,FullName Abbreviation Percentage 0...,0.25


In [44]:
def factorInWeights(row):
#     print(row.Name)
    row.Holdings['OverallPercentage']=row.Holdings.Percentage.apply(lambda v:float(v)*row.OverallWeight)
    return 

df.apply(factorInWeights,axis=1)

0    None
1    None
2    None
dtype: object

In [45]:
def appendFundName(row):
    row.Holdings['FundName']=row.Name
    return
df.apply(appendFundName,axis=1)

0    None
1    None
2    None
dtype: object

In [49]:
allHoldings=pd.concat(df.Holdings.values)
# allHoldings

In [65]:
grouped=allHoldings.groupby(
    ['Abbreviation']
).agg({
    'FullName':np.unique,
    'OverallPercentage': [np.sum,np.count_nonzero],
    'FundName':np.unique
}).rename(
    columns={
        'OverallPercentage':'Overall',
        'count_nonzero':'count',
        'unique':'values'
    }).reset_index()
# grouped

In [66]:
final=grouped.sort_values([('Overall', 'sum'),('Overall', 'count')],ascending=[0,0])
final.columns
final[['Abbreviation','FullName', 'FundName', 'Overall']]


Abbreviation                        FullName  \
                                         values   
23      MSFT:US                  Microsoft Corp   
18      GOOG:US                    Alphabet Inc   
8       AAPL:US                       Apple Inc   
7        700:HK            Tencent Holdings Ltd   
14       CRM:US              salesforce.com Inc   
13      BABA:US       Alibaba Group Holding Ltd   
10      ADBE:US               Adobe Systems Inc   
17        FB:US                    Facebook Inc   
11      AMZN:US                  Amazon.com Inc   
22      MCHP:US        Microchip Technology Inc   
4       2330:TT  Taiwan Semiconductor Manufactu   
12       APH:US                   Amphenol Corp   
19     GOOGL:US                    Alphabet Inc   
20      INTC:US                      Intel Corp   
27         V:US                        Visa Inc   
16      EUR:CUR                            EURO   
15      CSCO:US               Cisco Systems Inc   
0     068270:KS                   Celltrion Inc   
9       ABMD:US                     ABIOMED Inc   
1     084990:KS                  ViroMed Co Ltd   
26       SRX:AU              Sirtex Medical Ltd   
21        MA:US                  Mastercard Inc   
25      NVDA:US                     NVIDIA Corp   
2       1093:HK   CSPC Pharmaceutical Group Ltd   
3     207940:KS        Samsung Biologics Co Ltd   
6       4974:JP                  Takara Bio Inc   
5       3933:HK  United Laboratories Internatio   
24      MYGN:US             Myriad Genetics Inc   

                                             FundName Overall        
                                               values     sum count  
23  [Janus Henderson Global Technology Fund, Lyxor...  5.2000   2.0  
18  [Janus Henderson Global Technology Fund, Lyxor...  3.8550   2.0  
8            Lyxor MSCI World Information Technology   3.2575   1.0  
7              Janus Henderson Global Technology Fund  2.0950   1.0  
14             Janus Henderson Global Technology Fund  1.9100   1.0  
13             Janus Henderson Global Technology Fund  1.6200   1.0  
10             Janus Henderson Global Technology Fund  1.4950   1.0  
17           Lyxor MSCI World Information Technology   1.4950   1.0  
11             Janus Henderson Global Technology Fund  1.4750   1.0  
22             Janus Henderson Global Technology Fund  1.4650   1.0  
4              Janus Henderson Global Technology Fund  1.3450   1.0  
12             Janus Henderson Global Technology Fund  1.3100   1.0  
19           Lyxor MSCI World Information Technology   1.0925   1.0  
20           Lyxor MSCI World Information Technology   0.8650   1.0  
27           Lyxor MSCI World Information Technology   0.8075   1.0  
16            iShares Healthcare Innovation UCITS ETF  0.7400   1.0  
15           Lyxor MSCI World Information Technology   0.7275   1.0  
0             iShares Healthcare Innovation UCITS ETF  0.7100   1.0  
9             iShares Healthcare Innovation UCITS ETF  0.6975   1.0  
1             iShares Healthcare Innovation UCITS ETF  0.6450   1.0  
26            iShares Healthcare Innovation UCITS ETF  0.6275   1.0  
21           Lyxor MSCI World Information Technology   0.6075   1.0  
25           Lyxor MSCI World Information Technology   0.5050   1.0  
2             iShares Healthcare Innovation UCITS ETF  0.4950   1.0  
3             iShares Healthcare Innovation UCITS ETF  0.4650   1.0  
6             iShares Healthcare Innovation UCITS ETF  0.4425   1.0  
5             iShares Healthcare Innovation UCITS ETF  0.4375   1.0  
24            iShares Healthcare Innovation UCITS ETF  0.4200   1.0